<a href="https://colab.research.google.com/github/xboymsn/8tiles-problem-with-A-star-search/blob/main/Sentiment_Analysis_with_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
import torch

In [ ]:
!pip install transformers

In [ ]:
!pip install hazm

In [ ]:
!pip install tensorflow

In [5]:
import tensorflow as tf

In [6]:
import transformers as tr
from hazm import sent_tokenize

# Digikala Data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_csv('/content/drive/MyDrive/Data/preprocessed_digikala_dataset.csv')

In [9]:
batch_1 = df[:1000]

In [10]:
batch_1

,Comment,suggestion
0,'صدا نمیامد مرجوع دادم',0
1,'سلام من این گوشی رو یک ماه پیش از دیجی تهیه ک...,0
2,'دوستم این گوشی رو یک ماه پیش خرید.من یک هفته ...,0
3,'عالی است پیشنهادش میکنم برای هموطنانم',1
4,'به نسبت قیمت گوشی خوبی هست و از خرید خود راضی...,1
...,...,...
995,'عالی از تخفیفی که داده شد در دیجیکالا متشکرم',1
996,'با عرض سلام گوشی زیباییه فقط اشکال بزرگی که د...,0
997,'اولا ممنون از دیجی کالا برای تحویل به موقع و ...,1
998,'اصلا خراب بود ما پس دادیم',0


In [11]:
batch_1['suggestion'].value_counts()

1    521
0    479
Name: suggestion, dtype: int64

# Bert Model

In [ ]:
from transformers import AutoModel ,  AutoConfig, AutoTokenizer, TFAutoModel, AutoModelForSequenceClassification
import torch

model_name = "HooshvareLab/bert-fa-zwnj-base"
# model_name =  "bert-base-uncased"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert = AutoModel.from_pretrained(model_name)
bert.config.pad_token_id

In [13]:
# model_class, tokenizer_class, pretrained_weights = (tr.DistilBertModel, tr.DistilBertTokenizer, 'distilbert-base-uncased')


# # Load pretrained model/tokenizer
# tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
# model = model_class.from_pretrained(pretrained_weights)

با استفاده از توکنایزر برت جملات را به توکن هایش کاهش میدهیم و توکن اول و آخر را نیز اضافه میکنیم .

In [14]:
tokenized = batch_1['Comment'].apply(lambda x: tokenizer.encode(x , add_special_tokens=True))

In [15]:
tokenized.head()

0      [2, 114, 4134, 2204, 2859, 38368, 7445, 114, 3]
1    [2, 114, 3372, 1984, 1930, 2305, 1957, 1961, 2...
2    [2, 114, 15905, 1930, 2305, 1957, 1961, 2208, ...
3    [2, 114, 3686, 1933, 3444, 1121, 6294, 1959, 1...
4    [2, 114, 1923, 2503, 2395, 2305, 2852, 2065, 6...
Name: Comment, dtype: object

سایز بزرگترین جمله را پیدا میکنیم و دیگر جمله هارا به طول آن پد میکنیم .

In [16]:
max_len = 0
for i in tokenized.values:
  if len(i) > max_len :
      max_len = len(i)

max_len

127

In [17]:
padded = [i + [0]*(max_len-len(i)) for i in tokenized.values]

In [18]:
np.array(padded).shape

(1000, 127)

In [19]:
type(padded[0])

list

اتنشن را بر اساس صفر بودن یا نبودن پیدا میکنیم و ذخیره میکنیم .

In [20]:
attention_mask = np.where(np.array(padded) == 0, 0, 1)
attention_mask.shape

(1000, 127)

با استفاده از مدل و اینپوت آیدی های دریافت شده توسط توکنایزر و اتنشن بدست آمده از پدینگ وکتور 768 بعدی را بدست میاوریم.

In [21]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = bert(input_ids, attention_mask=attention_mask)

In [24]:
# input_ids = torch.tensor(np.array(padded))

# with torch.no_grad():
#   last_hidden_states = bert(input_ids)

In [22]:
Y = batch_1['suggestion']

In [23]:
X = last_hidden_states[0][:,0,:].numpy()

# Classifiers

در این بخش دیتاهای تست و آموزش را جدا میکنیم و به مدل مد نظر میدهیم.

In [25]:
from sklearn.metrics import confusion_matrix , accuracy_score , f1_score , precision_score , recall_score

In [26]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size = 0.2)

##Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train , Y_train)

# scores = cross_val_score(lr , X_train , Y_train)
# print("Logistic Regression classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
y_pred = lr.predict(X_test)
cm = confusion_matrix(Y_test,y_pred)
acc = accuracy_score(Y_test,y_pred)
precision = precision_score(Y_test,y_pred)
recall = recall_score(Y_test,y_pred)
f_measure = f1_score(Y_test,y_pred)
print(f"Confusion_matrix:\n{cm}\nAccuracy:{acc}\nPrecision:{precision}\nRecall:{recall}\nF_measure:{f_measure}\n*****************\n")

Confusion_matrix:
[[61 34]
 [29 76]]
Accuracy:0.685
Precision:0.6909090909090909
Recall:0.7238095238095238
F_measure:0.7069767441860465
*****************



In [34]:
comment_1 = "ظاهر شیک و رابط کاربری تر تمیزش منابسه . "
comment_2 = "با اینکه تصویر صفحه نمایش در ابتدا مناسب بود اما بعد از مدتی خراب شد و دیگر چیزی نمایش داده نشد ."

In [35]:
def predict(comment, model):

  text = comment
  comment = tokenizer.encode(comment , add_special_tokens = True)
  comment = [comment + [0]*(max_len-len(comment))]
  tokens = torch.tensor(np.array(comment))
  attentions = np.where(np.array(comment) == 0 , 0 , 1)
  attentions = torch.tensor(attentions)

  with torch.no_grad():
    comment = bert(tokens , attentions)

  comment = comment[0][:,0,:].numpy()

  if model.predict(comment) == 1:
    print(f"'{text}' is a positive comment .")
  else:
    print(f"'{text}' is a negative comment .")

In [37]:
predict(comment_1,lr)
predict(comment_2,lr)

'ظاهر شیک و رابط کاربری تر تمیزش منابسه . ' is a negative comment .
'با اینکه تصویر صفحه نمایش در ابتدا مناسب بود اما بعد از مدتی خراب شد و دیگر چیزی نمایش داده نشد .' is a negative comment .


##SVM

In [38]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train , Y_train)

SVC()

In [39]:
y_pred = svm.predict(X_test)
cm = confusion_matrix(Y_test,y_pred)
acc = accuracy_score(Y_test,y_pred)
precision = precision_score(Y_test,y_pred)
recall = recall_score(Y_test,y_pred)
f_measure = f1_score(Y_test,y_pred)
print(f"Confusion_matrix:\n{cm}\nAccuracy:{acc}\nPrecision:{precision}\nRecall:{recall}\nF_measure:{f_measure}\n*****************\n")

Confusion_matrix:
[[66 29]
 [32 73]]
Accuracy:0.695
Precision:0.7156862745098039
Recall:0.6952380952380952
F_measure:0.7053140096618358
*****************



In [40]:
predict(comment_1,svm)
predict(comment_2,svm)

'ظاهر شیک و رابط کاربری تر تمیزش منابسه . ' is a positive comment .
'با اینکه تصویر صفحه نمایش در ابتدا مناسب بود اما بعد از مدتی خراب شد و دیگر چیزی نمایش داده نشد .' is a negative comment .


##Neural Network

In [41]:
from sklearn.neural_network import MLPClassifier
neural_network = MLPClassifier(hidden_layer_sizes=(2000), learning_rate='adaptive', verbose=False)
neural_network.fit(X_train, Y_train)

MLPClassifier(hidden_layer_sizes=2000, learning_rate='adaptive')

In [42]:
y_pred = neural_network.predict(X_test)
cm = confusion_matrix(Y_test,y_pred)
acc = accuracy_score(Y_test,y_pred)
precision = precision_score(Y_test,y_pred)
recall = recall_score(Y_test,y_pred)
f_measure = f1_score(Y_test,y_pred)
print(f"Confusion_matrix:\n{cm}\nAccuracy:{acc}\nPrecision:{precision}\nRecall:{recall}\nF_measure:{f_measure}\n*****************\n")

Confusion_matrix:
[[63 32]
 [29 76]]
Accuracy:0.695
Precision:0.7037037037037037
Recall:0.7238095238095238
F_measure:0.7136150234741784
*****************



In [43]:
predict(comment_1,neural_network)
predict(comment_2,neural_network)

'ظاهر شیک و رابط کاربری تر تمیزش منابسه . ' is a positive comment .
'با اینکه تصویر صفحه نمایش در ابتدا مناسب بود اما بعد از مدتی خراب شد و دیگر چیزی نمایش داده نشد .' is a negative comment .


##Random Forest

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=2)
rf.fit(X_train,Y_train)
y_pred = rf.predict(X_test)

In [45]:
y_pred = rf.predict(X_test)
cm = confusion_matrix(Y_test,y_pred)
acc = accuracy_score(Y_test,y_pred)
precision = precision_score(Y_test,y_pred)
recall = recall_score(Y_test,y_pred)
f_measure = f1_score(Y_test,y_pred)
print(f"Confusion_matrix:\n{cm}\nAccuracy:{acc}\nPrecision:{precision}\nRecall:{recall}\nF_measure:{f_measure}\n*****************\n")

Confusion_matrix:
[[54 41]
 [29 76]]
Accuracy:0.65
Precision:0.6495726495726496
Recall:0.7238095238095238
F_measure:0.6846846846846847
*****************



In [46]:
predict(comment_1,rf)
predict(comment_2,rf)

'ظاهر شیک و رابط کاربری تر تمیزش منابسه . ' is a positive comment .
'با اینکه تصویر صفحه نمایش در ابتدا مناسب بود اما بعد از مدتی خراب شد و دیگر چیزی نمایش داده نشد .' is a positive comment .
